<a href="https://colab.research.google.com/github/KamilShape/emotion_recognition/blob/main/playwright_emotions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install playwright
!playwright install

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 46.8 MB/s eta 0:00:00
146.4 Mb [] 0% 0.0s146.4 Mb [] 0% 43.4s146.4 Mb [] 0% 35.3s146.4 Mb [] 0% 24.5s146.4 Mb [] 0% 16.3s146.4 Mb [] 1% 11.3s146.4 Mb [] 1% 7.9s146.4 Mb [] 2% 6.8s146.4 Mb [] 2% 5.6s146.4 Mb [] 3% 4.8s146.4 Mb [] 4% 4.3s146.4 Mb [] 5% 3.7s146.4 Mb [] 6% 3.2s146.4 Mb [] 7% 3.0s146.4 Mb [] 9% 2.6s146.4 Mb [] 10% 2.4s146.4 Mb [] 11% 2.3s146.4 Mb [] 13% 2.1s146.4 Mb [] 15% 1.9s146.4 Mb [] 16% 1.8s146.4 Mb [] 17% 1.8s146.4 Mb [] 19% 1.7s146.4 Mb [] 20% 1.6s146.4 Mb [] 21% 1.5s146.4 Mb [] 23% 1.4s146.4 Mb [] 25% 1.4s146.4 Mb [] 26% 1.3s146.4 Mb [] 27% 1.3s146.4 Mb [] 28% 1.3s146.4 Mb [] 29% 1.3s146.4 Mb [] 30% 1.2s146.4 Mb [] 31% 1.2s146.4 Mb [] 32% 1.2s146.4 Mb [] 33% 1.2s146.4 Mb [] 34% 1.2s146.4 Mb [] 36% 1.1s146.4 Mb [] 38% 1.1s146.4 Mb [] 40% 1.0s146.4 Mb [] 42% 0.9s146.4 Mb [] 44% 0.9s146.4 Mb [] 46% 0.9s146.4 Mb [] 47% 0.8s146.4 Mb [] 49% 0.8s146.4 Mb [] 51% 0.7s146.4 Mb [] 53% 0.7s146.4 Mb [] 54% 0.7s146.4 Mb 

In [3]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import random
import os
import numpy as np
from playwright.async_api import async_playwright

In [4]:
def match_span_class(tag):
    parent_span = tag.find_parent('span')
    if parent_span is not None:
        span_classes = parent_span.get('class', [])
        return any(cls.startswith('SummaryItemResponsive') for cls in span_classes)
    return False

In [7]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
faces = ['smiling-person', 'anger-person', 'disgust-person', 'fear-person', 'surprise-person' ]

In [17]:
data_path = 'drive/MyDrive/faces/data'

In [36]:
def get_image(photo_url, path, file_name, face):
  image_response = requests.get(photo_url)
  image_content = image_response.content
  img = Image.open(BytesIO(image_content))
  if img.mode == 'RGB':
    os.makedirs(f'{path}/{face}', exist_ok=True)
    img.save(f'{path}/{face}/{file_name}', 'png')

In [9]:
def get_photos(source, path, face):
  for image in source:
    http = image.find('https://')
    if http != -1:
      file_name = image.split('/')[-1]
      get_image(image, path, file_name, face)

In [40]:
#pages implicated by hand
images = []
for face in faces:
  modified_face = face.replace('-', '%20')
  async def run():
    async with async_playwright() as p:
      browser = await p.chromium.launch()
      page = await browser.new_page()
      await page.goto(f'https://www.istockphoto.com/pl/search/2/image?page=12&phrase={modified_face}') #input pages from 1-100
      # Get the page content
      content = await page.content()
      await browser.close()
      # Use Beautiful Soup to parse the HTML
      soup = BeautifulSoup(content, 'html.parser')
      # Find all image tags and add their src attributes to the image_sources list
      all_images = soup.find_all('img')
      images_in_span = soup.find_all(lambda tag: tag.name == 'img' and match_span_class(tag))
      interesting_images = [img for img in all_images if img not in images_in_span]
      for img in interesting_images:
        src = img.get('src')
        images.append(src)
      print(images)
      get_photos(images, data_path, face)
      print(f"{len(os.listdir(f'{data_path}/{face}'))} photos in {face} folder")
  images = []
  await run()

['/search/assets/static/color_mood_all-426fbeb2abd7da282b67.jpg', 'https://media.istockphoto.com/id/1363697779/pl/zdj%C4%99cie/anonimowy-m%C4%99%C5%BCczyzna-u%C5%BCywa-laptopa-do-handlu-online-podczas-pracy-w-domu.jpg?s=612x612&w=0&k=20&c=-t23SJXbo8cOM7S6cwMrrsR1kQug78MXSVhbFquIN5o=', 'https://media.istockphoto.com/id/1199653718/pl/zdj%C4%99cie/zarz%C4%85dzanie-klientami-%C5%82atwiejsze-dzi%C4%99ki-technologii-mobilnej.jpg?s=612x612&w=0&k=20&c=JGwul7vaH7KnuUssp4GN9NJTld1RkT0kVfYRJbpyg-s=', 'https://media.istockphoto.com/id/638898084/pl/zdj%C4%99cie/dziewczyna-przy-u%C5%BCyciu-smartphone.jpg?s=612x612&w=0&k=20&c=2kvAKI8LzNJOnMcISxlyzuRv7_jqRN11yTgvPVEW99k=', 'https://media.istockphoto.com/id/1450344486/pl/zdj%C4%99cie/okno-okulary-i-m%C5%82oda-kobieta-z-u%C5%9Bmiechem-relaks-i-pewno%C5%9B%C4%87-siebie-w-biurze-biznes-latynoska.jpg?s=612x612&w=0&k=20&c=T_PROLOFNxRcMpA4jvtyVfKgeMoLaofFDJ_96OzOUfE=', 'https://media.istockphoto.com/id/1057500046/pl/zdj%C4%99cie/przyjacielski-czat.jpg?s=612x